In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [6]:
promote = pd.read_csv('promoted_content.csv')

In [7]:
chunksize=50000

In [8]:
train = pd.read_csv('clicks_train.csv',iterator=True,chunksize=chunksize)

In [11]:
for chunk in train:
    chunk=pd.merge(chunk,promote,how='left',on='ad_id')
    predictors=[x for x in chunk.columns if x not in ['display_id','clicked']]
    alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
    alg.fit(chunk[predictors], chunk["clicked"])#Fit the Algorithm

In [19]:
test = pd.read_csv('clicks_test.csv',iterator=True,chunksize=chunksize)

In [20]:
predY=[]

In [21]:
for chunk in test:
    init_chunk_size=len(chunk)
    chunk=pd.merge(chunk,promote,how='left',on='ad_id')
    chunk=chunk.fillna(0.0)
    chunk_pred=list(alg.predict_proba(chunk[predictors]).astype(float)[:,1])
    predY += chunk_pred

In [25]:
print chunk

       display_id   ad_id  document_id  campaign_id  advertiser_id
0        23115203  281177      1522802        24152           1650
1        23115203  370035      1827718        30017           2352
2        23115203  436515      1472428        22477            408
3        23115203  529119      2642676        33878           2179
4        23115204    8020       498497         1559            498
5        23115204   49770       861670         6681           1913
6        23115204  149665       689384        18998           1008
7        23115204  215844      1377637        24413           1863
8        23115204  377666       691081        30214           3251
9        23115205   47933       358646         6632            784
10       23115205  100945      1050787        13140            407
11       23115205  158344       984694        20051           2151
12       23115205  439461      2121705        31380           4200
13       23115206  117000      1129971        15009           

In [26]:
test = pd.read_csv('clicks_test.csv')

In [43]:
results=pd.concat((test,pd.DataFrame(predY)) ,axis=1,ignore_index=True)

In [45]:
results.columns = ['display_id','ad_id','clicked']

In [46]:
results = results.sort_values(by=['display_id','clicked'], ascending=[True, False])

In [47]:
results = results.reset_index(drop=True)

In [49]:
results=results.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [55]:
results.to_csv('random_forest_promote.csv', index=False)

In [56]:
results

,display_id,ad_id
0,16874594,172888 180797 66758 162754 150083 170392
1,16874595,8846 30609 143982
2,16874596,289122 57197 11430 289915 173005 132820 153260...
3,16874597,305790 308836 155945 285834 137858 143981 1809...
4,16874598,335632 145937 67292 250082
5,16874599,173130 213116 91681 163776 296295 210516
6,16874600,2150 70529 133050 30682 114836 57591
7,16874601,129490 140942 92003 118470 14082 190713
8,16874602,131316 154918 268548 269017 281563
9,16874603,163139 156050 140423 44460
